In [ ]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense
from keras import applications

# image dimensions
img_width, img_height = 150, 150

top_model_weights_path = 'bottleneck_fc_model.h5'
train_data_dir = 'data/train'
validation_data_dir = 'data/validation'
nb_train_samples = 2000
nb_validation_samples = 800
epochs = 50
batch_size = 16


def save_bottlebeck_features():
    datagen = ImageDataGenerator(rescale=1. / 255)

    # build the VGG16 network
    model = applications.VGG16(include_top=False, weights='imagenet')

    generator = datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)
    bottleneck_features_train = model.predict_generator(
        generator, nb_train_samples // batch_size)
    np.save(open('bottleneck_features_train.npy', 'wb'),
            bottleneck_features_train)

    generator = datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)
    bottleneck_features_validation = model.predict_generator(
        generator, nb_validation_samples // batch_size)
    np.save(open('bottleneck_features_validation.npy', 'wb'),
            bottleneck_features_validation)
    

In [ ]:
save_bottlebeck_features()

In [25]:
def train_top_model():
    train_data = np.load(open('bottleneck_features_train.npy', 'rb'))
    train_labels = np.array(
        [0] * 1000 + [1] * 1000)

    validation_data = np.load(open('bottleneck_features_validation.npy', 'rb'))
    validation_labels = np.array(
        [0] * 400 + [1] * 400)

    model = Sequential()
    model.add(Flatten(input_shape=train_data.shape[1:]))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer='rmsprop',
                  loss='binary_crossentropy', metrics=['accuracy'])

    model.fit(train_data, train_labels,
              epochs=epochs,
              batch_size=batch_size,
              validation_data=(validation_data, validation_labels))
    model.save_weights(top_model_weights_path)


In [26]:
train_top_model()

Train on 2000 samples, validate on 800 samples
Epoch 1/50
2000/2000 [==============================] - 3s 2ms/step - loss: 0.6418 - acc: 0.7675 - val_loss: 0.4174 - val_acc: 0.7900
Epoch 2/50
2000/2000 [==============================] - 3s 2ms/step - loss: 0.3739 - acc: 0.8470 - val_loss: 0.4794 - val_acc: 0.7913
Epoch 3/50
2000/2000 [==============================] - 3s 2ms/step - loss: 0.3198 - acc: 0.8760 - val_loss: 0.4240 - val_acc: 0.8425
Epoch 4/50
2000/2000 [==============================] - 3s 2ms/step - loss: 0.2748 - acc: 0.8955 - val_loss: 0.2180 - val_acc: 0.9050
Epoch 5/50
2000/2000 [==============================] - 3s 2ms/step - loss: 0.2323 - acc: 0.9105 - val_loss: 0.2454 - val_acc: 0.9100
Epoch 6/50
2000/2000 [==============================] - 3s 2ms/step - loss: 0.2162 - acc: 0.9185 - val_loss: 0.2303 - val_acc: 0.9038
Epoch 7/50
2000/2000 [==============================] - 3s 2ms/step - loss: 0.1890 - acc: 0.9320 - val_loss: 0.6226 - val_acc: 0.8350
Epoch 8/50
2000

### Achieved accuracy of ~90% with bottleneck features of pretrained network